In [1]:
import os.path
import sys
import pandas as pd
import numpy as np
import yaml
import re
import subprocess
import io
from datetime import datetime, date

In [2]:
report = '/users/lvelten/project/Methylome/analysis/selection_pipeline/RnBeads/rnb_report_20211006_reduced_differential/'
output = '/users/lvelten/project/Methylome/analysis/selection_pipeline/RnBeads/DMRs/'
config_file = '/users/lvelten/project/Methylome/src/selection_pipeline/config.yaml'
script_path = '/users/lvelten/project/Methylome/src/selection_pipeline/check_cut_site.R'

In [3]:
with open(config_file) as conf: 
    config = yaml.safe_load(conf)

In [4]:
def check_cut_sites(frame, out, config, n):
    print(str(datetime.now()) + ': Checking for enzyme cutsite')
    f = out + '/temp.csv'
    frame.to_csv(f)
    cmd = "Rscript " + script_path + " -i " + f + " -c " + config + " -n " + str(n)
    proc = subprocess.run(cmd,shell=True)
    res = pd.read_csv(f)
    return res

In [5]:
def return_reliable_DMCs(rnbOut, n, existing_dmcs, out, config):
    print(str(datetime.now()) + ': Start selecting reliable DMCs')
    diff_table = pd.read_csv(rnbOut)
    row_names = np.array(diff_table['Chromosome'] + ':' + [str(n) for n in diff_table['Start']])
    diff_table = diff_table.loc[[n not in existing_dmcs for n in row_names],]
    diff_table = diff_table.sort_values('combinedRank')
    group_names = np.array(diff_table.columns[['sd.' in n for n in diff_table.columns]].array)
    group_names = [re.sub('sd.','',n) for n in group_names]
    print('Processing diffTable for ' + group_names[0] + ' and ' + group_names[1])
    sds_first = diff_table['sd.'+group_names[0]]
    sds_second = diff_table['sd.'+group_names[1]]
    se_sd_first = np.std(sds_first)/np.sqrt(len(sds_first))
    se_sd_second = np.std(sds_second)/np.sqrt(len(sds_second))
    low_sds = np.logical_and(sds_first<(np.mean(sds_first)+2*se_sd_first),sds_second<(np.mean(sds_second)+2*se_sd_second))
    diff_table = diff_table.loc[low_sds,]
    diff_positive = diff_table.loc[diff_table['mean.diff']>0,]
    diff_positive['type'] = 'low_' + group_names[1]
    diff_positive = check_cut_sites(diff_positive, out, config, n//2)
    diff_negative = diff_table.loc[diff_table['mean.diff']<0,]
    diff_negative['type'] = 'low_' + group_names[0]
    diff_negative = check_cut_sites(diff_negative, out, config, n//2)
    diff_positive['comparison'] = group_names[0] + ' vs. ' + group_names[1]
    diff_negative['comparison'] = group_names[0] + ' vs. ' + group_names[1]
    print(str(datetime.now()) + ': Completed selecting reliable DMCs')
    return(pd.concat([diff_positive,diff_negative]))

In [6]:
diff_report = report + '/differential_methylation_data/'
diff_results = np.array(os.listdir(diff_report))
diff_results = np.array(diff_results[['diffMethTable_site' in n for n in diff_results]])
num_dmcs = config['dmcs']['number']//len(diff_results)
res_all = pd.DataFrame()
if 'amplicon_blacklist' in config['dmcs'].keys():
    blacklist = pd.read_csv(config['dmcs']['amplicon_blacklist'], sep='\t')
    ids = blacklist['Chromosome'] + ':' + [str(n) for n in blacklist['Start']]

else:
    ids = np.array("")

new_dmrs = None
for diff_res in diff_results:
    if new_dmrs is not None:
        ids = np.append(ids,new_dmrs)
        
    else:
        ids = new_dmrs
        
    res = return_reliable_DMCs(diff_report + diff_res, num_dmcs, np.array(ids), output, config_file)
    res_all = pd.concat([res_all,res])
    new_dmrs = np.array(res_all['Chromosome'] + ':' + [str(n) for n in res_all['Start']])

None
2021-10-19 10:45:31.454983: Start selecting reliable DMCs
Processing diffTable for MPP and MPP2


/users/mscherer/software/anaconda3/envs/selection/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2021-10-19 10:52:54.163253: Checking for enzyme cutsite


/users/mscherer/software/anaconda3/envs/selection/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2021-10-19 10:57:45.235663: Checking for enzyme cutsite
2021-10-19 11:01:35.757100: Completed selecting reliable DMCs
[None 'chr12:75814671' 'chr7:80417258' 'chr8:23167651' 'chr1:87170094'
 'chr11:60160091' 'chr17:87826529' 'chr17:29795612' 'chr8:69889427'
 'chr6:147263223' 'chr9:21797959' 'chr1:168226411' 'chr8:120484638'
 'chr6:85051215' 'chr6:122763090' 'chr19:16141937' 'chr1:63953638'
 'chr6:115770179' 'chr4:154928565' 'chr18:24101840' 'chr6:67534306'
 'chr6:124686328' 'chr4:115849162' 'chr13:96038077' 'chr6:140509399'
 'chr8:115534210' 'chr10:62136580' 'chr10:117658659' 'chr4:155173782'
 'chr5:147292593' 'chr1:189867763' 'chr10:79612767' 'chr2:30787709'
 'chr8:122322917' 'chr10:127566277' 'chr10:60536699' 'chr3:84468207'
 'chr8:122323065' 'chr2:128092245' 'chr14:74972308' 'chr7:79313834'
 'chr1:193156489' 'chr10:81393940' 'chr18:64346595' 'chr3:122340606'
 'chr16:6349444' 'chr12:8829857' 'chr10:127163164' 'chr3:101746126'
 'chr10:125839592' 'chr4:147275697' 'chr14:58383162' 'chr17

...

2021-10-19 12:23:53.364528: Checking for enzyme cutsite
2021-10-19 12:28:12.116237: Completed selecting reliable DMCs
[None 'chr12:75814671' 'chr7:80417258' ... 'chr18:58713930'
 'chr9:43194077' 'chr10:5317793']
2021-10-19 12:28:12.144382: Start selecting reliable DMCs
Processing diffTable for MPP1 and MPP2


/users/mscherer/software/anaconda3/envs/selection/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2021-10-19 12:51:21.874202: Checking for enzyme cutsite


/users/mscherer/software/anaconda3/envs/selection/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2021-10-19 12:56:17.328438: Checking for enzyme cutsite
2021-10-19 13:00:15.661505: Completed selecting reliable DMCs


In [7]:
        np.append(ids,new_dmrs)


array([None, 'chr12:75814671', 'chr7:80417258', ..., 'chr12:113113333',
       'chr15:59454576', 'chr1:180265312'], dtype=object)

In [8]:
config = yaml.safe_load('/users/lvelten/project/Methylome/src/selection_pipeline/config.yaml')

In [9]:
res_all.to_csv(output + 'all_DMRs.csv')